# 3 - Détection des Opérations Juridiques

Dans cet atelier, vous allez détecter automatiquement des **opérations juridiques** dans un arrêté préfectoral à l'aide d'un **LLM (Large Language Model)**.

## Contexte

Un arrêté préfectoral peut **modifier**, **abroger** ou **compléter** une partie d'un arrêté antérieur, ainsi que l'arrêté entier. Par exemple :
- **Abrogation** : "L'arrêté du 2023-05-10 est abrogé".
- **Modification** : "L'article 2.1 de l'arrêté du 2023-05-10 est remplacé par les dispositions suivantes..."
- **Ajout** : "Il est ajouté à l'arrêté du 2023-05-10 un article 5 ainsi rédigé..."

L'objectif est d'**extraire automatiquement** ces opérations sous forme structurée pour pouvoir ensuite reconstituer l'état consolidé d'un permis.

## Plan du TP

0. **Exercice découverte** : détecter des opérations avec un prompt libre
1. **Définir la classe `Operation`** : structure de données pour représenter une opération
2. **Prompt Engineering** : concevoir un prompt structuré pour détecter les opérations
3. **Subtarget Parsing** : identifier précisément quelle partie d'un article est modifiée

---

## 0. Exercice découverte : détecter des opérations juridiques

Avant de définir des structures de données, commençons par **comprendre le problème**.

Vous allez recevoir des extraits d'arrêtés préfectoraux qui modifient des arrêtés antérieurs. Votre mission : écrire un prompt pour qu'un LLM détecte automatiquement ces modifications.

### 0.1 Exemples d'arrêtés

Voici 4 extraits d'arrêtés modificateurs :

In [3]:
# Exemple 1 : ABROGATION complète
exemple_1 = """
Extrait de l'arrêté du 8 janvier 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'arrêté préfectoral du 5 oct. 2023 est abrogé.</p>
</section>
"""

# Exemple 2 : MODIFICATION d'un article
exemple_2 = """
Extrait de l'arrêté du 12 septembre 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>Les dispositions de l'article 2.1 de l'arrêté du 5 octobre 2023 sont abrogées et remplacées par le contenu suivant :</p>
    <p>Le débit maximal de prélèvement d'eau est fixé à 150 m³/h.</p>
</section>
"""

# Exemple 3 : AJOUT d'un nouvel article
exemple_3 = """
Extrait de l'arrêté du 23/01/2023 :
<section data-spec="section" data-number="2">
    <h2>Article 2</h2>
    <p>Il est ajouté à l'arrêté du 2022-12-15 un article 4.3 ainsi rédigé :</p>
    <p>L'exploitant doit installer un dispositif de mesure du débit avant le 31 décembre 2026.</p>
</section>
"""

# Exemple 4 : MODIFICATION d'une partie spécifique
exemple_4 = """
Extrait de l'arrêté du 12 septembre 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>Le tableau de l'article 3 de l'arrêté du 5 octobre 2023 est remplacé par le tableau suivant :</p>
    <table>
        <tr><th>Paramètre</th><th>Valeur limite</th></tr>
        <tr><td>pH</td><td>6.5 - 8.5</td></tr>
        <tr><td>DCO</td><td>125 mg/L</td></tr>
    </table>
</section>
"""


# Exemple 5 : Plusieurs opérations dans un extrait 
exemple_5 = """
Extrait de l'arrêté du 4 mars 2025 :
<section data-spec="section" data-number="1">
    <h2>Article 4</h2>
    <p>Les dispositions suivantes de l'arrêté du 12 septembre 2024 sont abrogées :</p>
    <ul>
        <li> - la première ligne de l' article 5.3.3; </li>
        <li> - l'article 6.7 intitulé «Déchets produits par l'établissement »; </li>
        <li> - la colonne n°1 du tableau de l'article 3.4. </li>
    </ul>
</section>
"""

print("5 exemples chargés")

5 exemples chargés


### 0.2 Exercice : Écrire un prompt libre

**Votre mission :** Écrivez un prompt qui demande au LLM de détecter les opérations juridiques dans ces extraits.

**Pour l'instant, ne vous préoccupez pas du format de sortie** - écrivez simplement ce qui vous semble naturel.

In [4]:
def mon_premier_prompt(html_content: str) -> str:
    """
    Votre prompt libre pour détecter les opérations.
    """
    # TODO : Écrivez votre prompt ici
    prompt = f"""
Voici un extrait d'arrêté :
{html_content}
Détecte moi les opérations juridiques définies dans cet extrait. 
"""
    return prompt

# Testez sur l'exemple 1
print(mon_premier_prompt(exemple_1))


Voici un extrait d'arrêté :

Extrait de l'arrêté du 8 janvier 2024 :
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'arrêté préfectoral du 5 oct. 2023 est abrogé.</p>
</section>

Détecte moi les opérations juridiques définies dans cet extrait. 



### 0.3 Tester avec le LLM

Maintenant, testons votre prompt avec un vrai LLM (Mistral AI).

**Configuration :** Assurez-vous d'avoir une clé API dans un fichier `.env` :
```
MISTRAL_API_KEY=votre_clé_ici
```

In [5]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()
if os.getenv("MISTRAL_API_KEY"):
    client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))
    print("✓ LLM configuré")


def call_llm(prompt: str, model: str = "mistral-large-latest") -> str:
    """Appelle l'API Mistral."""
    response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": prompt, "temperature": 0}],
    )
    return response.choices[0].message.content

✓ LLM configuré


In [6]:
# Testez votre prompt sur les 4 exemples
print("=" * 60)
print("EXEMPLE 1 - Abrogation")
print("=" * 60)
response_1 = call_llm(mon_premier_prompt(exemple_1))
print(response_1)

print("\n" + "=" * 60)
print("EXEMPLE 2 - Modification")
print("=" * 60)
response_2 = call_llm(mon_premier_prompt(exemple_2))
print(response_2)

print("\n" + "=" * 60)
print("EXEMPLE 3 - Ajout")
print("=" * 60)
response_3 = call_llm(mon_premier_prompt(exemple_3))
print(response_3)

print("\n" + "=" * 60)
print("EXEMPLE 4 - Modification tableau")
print("=" * 60)
response_4 = call_llm(mon_premier_prompt(exemple_4))
print(response_4)

print("\n" + "=" * 60)
print("EXEMPLE 4 - Modification tableau")
print("=" * 60)
response_5 = call_llm(mon_premier_prompt(exemple_5))
print(response_5)

EXEMPLE 1 - Abrogation
Dans cet extrait de l'arrêté du 8 janvier 2024, l'**opération juridique** principale est une **abrogation**.

### Détails de l'opération juridique :
1. **Abrogation** :
   - L'**article 1** de l'arrêté du 8 janvier 2024 **abroge** (supprime) l'arrêté préfectoral du 5 octobre 2023.
   - L'abrogation est une opération juridique qui met fin à l'application d'un texte antérieur, sans effet rétroactif (contrairement à l'annulation). Elle prend effet à compter de la date de publication du nouvel arrêté (ici, le 8 janvier 2024).

### Autres éléments à noter :
- **Nature de l'acte** : Il s'agit d'un acte administratif unilatéral (arrêté préfectoral) modifiant l'ordonnancement juridique en supprimant un texte antérieur.
- **Effet** : L'arrêté du 5 octobre 2023 cesse de produire ses effets pour l'avenir, mais les actes pris en application de ce dernier avant son abrogation peuvent rester valables (sauf disposition contraire).

Aucune autre opération juridique (comme une mo

### 0.4 Constat

**Observez les résultats :** Les réponses du LLM sont probablement :
- ✓ Correctes sur le fond
- ✗ **Différentes en format** (phrases, listes, structures variables)
- ✗ **Difficiles à parser automatiquement** dans un programme

**Problème :** Pour traiter automatiquement ces opérations (par exemple, appliquer les modifications pour reconstituer l'arrêté consolidé), on a besoin d'un **format structuré et prévisible**.

**Solution :** Définir des objets Python (classes) qui représentent ces opérations, puis imposer au LLM de respecter ce format (JSON).

---

## 1. Définition de la classe `Operation`

Une opération juridique contient plusieurs informations :
- **Type** : ADD, REMOVE, REPLACE
- **Source** : l'article de l'arrêté modificateur qui contient l'opération
- **Target** : l'arrêté et l'article à modifier
- **Operand** : le nouveau contenu (pour ADD et REPLACE)
- **SubTarget** : la partie précise à modifier (ex: "première phrase", "le tableau")

### Exercice 1.1 : Définir les types d'opérations

Créez une énumération `OperationType` avec les valeurs : `ADD`, `REMOVE`, `REPLACE`

### Exercice 1.2 : Définir la classe `NodeId`

Un `NodeId` identifie de manière unique un article dans un arrêté. Il contient :
- `arrete_id` : la date de l'arrêté au format "YYYY-MM-DD" (ex: "2023-05-10")
- `article_id` : le numéro de l'article (ex: "2.1.3", "ALL", "NEW:3.4")

NB : Les cas "ALL" et "NEW:" correspondent respectivement au cas où une opération concerne tous les articles d'un arrêté (pour une abrogation de l'arrêté entier par exemple), et le cas où l'article cible n'existe pas encore (lors de la création d'un article). 

Complétez la classe ci-dessous :

Utilisez la cellule suivante pour tester directement votre code.

In [16]:
try: 
    node_invalid = NodeId(arrete_id="2023-15-10", article_id="deux")  # Doit lever une erreur de format
except ValueError as e:
    print(f"Erreur attendue capturée : {e}")

Erreur attendue capturée : 2 validation errors for NodeId
arrete_id
  Value error, Date invalide dans arrete_id: '2023-15-10' [type=value_error, input_value='2023-15-10', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
article_id
  Value error, article_id doit être au format numérique (ex: '1.2', '3.1.4'), ALL ou NEW:X reçu: 'deux' [type=value_error, input_value='deux', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


### Exercice 1.3 : Définir la classe `Operation`

A partir de la définition de l'opération ci-dessus et en utilisant les types OperationType et NodeId créés ci-dessus,

complétez la classe ci-dessous en ajoutant tous les champs nécessaires :

Utilisez la cellule suivante pour tester directement votre code.

In [20]:
operation = Operation(
     id="op_004",
     source_id=NodeId(arrete_id="2024-09-12", article_id="3.1"),
     target_id=NodeId(arrete_id="2023-05-10", article_id="2.1"),
     operation_type=OperationType.REPLACE,
     operand="<table> <tr><th>Paramètre</th><th>Valeur limite</th></tr><tr><td>pH</td><td>6.5 - 8.5</td></tr><tr><td>DCO</td><td>125 mg/L</td></tr> </table>",
     sub_target="le tableau"
)
print(operation.model_dump_json(indent=2))

{
  "id": "op_004",
  "source_id": {
    "arrete_id": "2024-09-12",
    "article_id": "3.1"
  },
  "target_id": {
    "arrete_id": "2023-05-10",
    "article_id": "2.1"
  },
  "operation_type": "REPLACE",
  "operand": "<table> <tr><th>Paramètre</th><th>Valeur limite</th></tr><tr><td>pH</td><td>6.5 - 8.5</td></tr><tr><td>DCO</td><td>125 mg/L</td></tr> </table>",
  "sub_target": "le tableau"
}


In [21]:
# Vos réponses ici
reponses = {
    "operation_type": "REPLACE",  # À compléter
    "source_article": "1",
    "target_arrete": "2023-10-05",
    "target_article": "2.1",
    "operand": "<p>Le débit maximal de prélèvement d'eau est fixé à 150 m³/h.</p>"
}

print(reponses)

{'operation_type': 'REPLACE', 'source_article': '1', 'target_arrete': '2023-10-05', 'target_article': '2.1', 'operand': "<p>Le débit maximal de prélèvement d'eau est fixé à 150 m³/h.</p>"}


In [ ]:
def create_detection_prompt(html_content: str) -> str:
    """
    Crée un prompt pour détecter les opérations juridiques dans un extrait HTML.
    
    Args:
        html_content: Le contenu HTML de l'arrêté
        
    Returns:
        Le prompt complet à envoyer au LLM
    """
    prompt = f"""
Voici un extrait HTML d'arrêté préfectoral :
\"\"\"{html_content}\"\"\"

Ta tâche : identifier toutes les opérations juridiques (modifications, ajouts, abrogations).

# TODO : Compléter le prompt avec :
# - Instructions claires
# - Format JSON attendu
# - Exemples pour chaque type d'opération
# - Consignes sur les cas limites

Réponds avec une liste JSON uniquement.
"""
    return prompt

# Test du prompt
prompt = create_detection_prompt(exemple_html)
print(prompt[:500] + "...")



Voici un extrait HTML d'arrêté préfectoral :
"""
<section data-spec="section" data-number="1">
    <h2>Article 1</h2>
    <p>L'article 2.1 de l'arrêté du 2023-05-10 est modifié comme suit :</p>
    <p>Le débit maximal autorisé est porté à 150 m³/h.</p>
</section>
"""

Ta tâche : identifier toutes les opérations juridiques (modifications, ajouts, abrogations).

# TODO : Compléter le prompt avec :
# - Instructions claires
# - Format JSON attendu
# - Exemples pour chaque type d'opération
# - Consi...


In [9]:
import os
from dotenv import load_dotenv
from mistralai import Mistral
import json

# Charger les variables d'environnement
load_dotenv()

# Initialiser le client Mistral
client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

def call_llm(prompt: str, model: str = "mistral-large-latest") -> str:
    """
    Appelle l'API Mistral avec le prompt donné.
    
    Args:
        prompt: Le prompt à envoyer
        model: Le modèle à utiliser
        
    Returns:
        La réponse du LLM
    """
    response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Test
try:
    prompt = create_detection_prompt(exemple_html)
    response = call_llm(prompt)
    print("Réponse du LLM :")
    print(response)
    
    # Parser le JSON
    operations = json.loads(response)
    print(f"\n✓ {len(operations)} opération(s) détectée(s)")
    
except Exception as e:
    print(f"Erreur : {e}")


Erreur : [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée


In [ ]:
import json

# Sorties attendues pour validation
expected_outputs = {
    "exemple_1": [
        {
            "operation_type": "REMOVE",
            "source_arrete": "2024-01-08",
            "source_article": "1",
            "target_arrete": "2023-05-10",
            "target_article": "ALL"
        }
    ],
    "exemple_2": [
        {
            "operation_type": "REPLACE",
            "source_arrete": "2024-09-12",
            "source_article": "1",
            "target_arrete": "2023-05-10",
            "target_article": "2.1",
            "operand" : "<p>Le débit maximal de prélèvement d'eau est fixé à 150 m³/h.</p>", 
        }
    ],
    "exemple_3": [
        {
            "operation_type": "ADD",
            "source_arrete": "2023-01-23",
            "source_article": "2",
            "target_arrete": "2022-12-15",
            "target_article": "NEW:4.3",
            "operand": "<p>L'exploitant doit installer un dispositif de mesure du débit avant le 31 décembre 2026.</p>"
        }
    ],
    "exemple_4": [
        {
            "operation_type": "REPLACE",
            "source_arrete": "2024-09-12",
            "source_article": "1",
            "target_arrete": "2023-05-10",
            "target_article": "3",
            "sub_target": "le tableau",
            "operand": "<table> <tr><th>Paramètre</th><th>Valeur limite</th></tr> <tr><td>pH</td><td>6.5 - 8.5</td></tr> <tr><td>DCO</td><td>125 mg/L</td></tr>  </table>",
        }
    ]
}

print("✓ Sorties attendues chargées")

In [ ]:
# Validation automatique des résultats
def valider_detection(resultats, expected):
    """Compare les résultats avec les sorties attendues."""
    score = 0
    total = len(expected)
    
    for nom, expected_ops in expected.items():
        if nom not in resultats:
            print(f"✗ {nom} : non testé")
            continue
            
        result_ops = resultats[nom]
        
        if len(result_ops) != len(expected_ops):
            print(f"✗ {nom} : {len(result_ops)} opération(s) au lieu de {len(expected_ops)}")
            continue
        
        # Vérifier les champs essentiels
        op_result = result_ops[0]
        op_expected = expected_ops[0]
        
        checks = {
            "operation_type": op_result.get("operation_type") == op_expected["operation_type"],
            "target_arrete": op_result.get("target_arrete") == op_expected["target_arrete"],
            "target_article": op_result.get("target_article") == op_expected["target_article"],
        }
        
        if all(checks.values()):
            print(f"✓ {nom} : correct")
            score += 1
        else:
            errors = [k for k, v in checks.items() if not v]
            print(f"✗ {nom} : erreurs dans {errors}")
    
    print(f"\n{'='*60}")
    print(f"Score : {score}/{total}")
    if score == total:
        print("🎉 Parfait ! Votre prompt détecte correctement toutes les opérations.")
    elif score >= total * 0.75:
        print("👍 Bon travail ! Quelques ajustements à faire.")
    else:
        print("💡 Continuez à améliorer votre prompt.")
    
    return score == total

valider_detection(resultats, expected_outputs)

## 3. Subtarget Parsing

Le **subtarget** permet d'identifier précisément **quelle partie** d'un article doit être modifiée.

Exemples :
- "la première phrase"
- "le tableau"
- "le deuxième alinéa"
- "la colonne N°2 du tableau"


### 3.1 Définir les types de subtargets


### Exercice 3.1 : Parser un subtarget avec regex

Créez une fonction qui parse des descriptions simples de subtargets.

**Exemples à gérer :**
- "première phrase" → PHRASE, position=1
- "le tableau" → TABLEAU
- "deuxième alinéa" → ALINEA, position=2
- "dernière ligne du tableau" → LIGNE_TABLEAU, position=0


### 3.2 Intégration avec le prompt

Maintenant, intégrez la détection de subtarget dans le prompt. Le LLM doit retourner un champ `sub_target` avec la description.

Modifiez votre prompt pour inclure des exemples de subtargets :

---

## 5. Pipeline complet

Assemblons tous les éléments pour créer le pipeline complet de détection d'opérations.

---

## 6. Exercices avancés

### 6.1 Gestion des erreurs

Améliorez le pipeline pour gérer :
- Les réponses JSON malformées du LLM
- Les marqueurs non trouvés
- Les formats de date invalides
- Les articles inexistants

### 6.2 Métriques et validation

Créez des fonctions pour :
- Valider le format d'une opération
- Calculer des métriques (précision, rappel)
- Comparer avec des annotations manuelles

### 6.3 Optimisation du prompt

Testez différentes variations du prompt et comparez :
- Température du LLM
- Longueur du prompt
- Nombre d'exemples
- Format des instructions

Créez un notebook de comparaison :

In [ ]:
# TODO : Expérimentations sur le prompt

---

## Conclusion

Vous avez maintenant un pipeline complet pour :
1. ✓ Définir une structure de données pour les opérations
2. ✓ Utiliser un LLM pour détecter les opérations
3. ✓ Parser les subtargets avec des regex
4. ✓ Extraire les operands avec des marqueurs

**Prochaines étapes :**
- Améliorer la robustesse (gestion d'erreurs)
- Optimiser le prompt (expérimentations)
- Valider avec des données réelles
- Intégrer dans le pipeline de consolidation complet

Bon travail ! 🎉